# SEP Distribution Tool

This tool downloads SEP intensity-time series data from various different spacecraft and visualizes the SEP distribution using Gaussian curves in one final results plot.

This tool uses a preset proton energy range to keep the various observations comparable.

There is an option to also show a spacecraft-constellation plot (Solar-MACH) and a table summarizing the spacecraft coordinates for the selected time interval.

In [1]:
# Import modules
import JTL_SEP_functions as jtl

from seppy.util import jupyterhub_data_path
import datetime as dt
import numpy as np
import os
import pandas as pd

from solarmach import SolarMACH

## Saving figures and date

You can usually save a figure from the Notebook by right-clicking on it while holding down the ⇧ Shift key, then select "Save Image As..." (or similar).

In [6]:
# Set your local path where you want to save the data files. 
# If run on the project's JupyterHub server, set it to a common data folder. 
data_path = f"{os.getcwd()}{os.sep}data/"
data_path = jupyterhub_data_path(data_path)

## Define the event details

Collect the event start and end dates (specifying the start at / near the observed flare onset time), and the eruption location (in Stonyhurst).

In [7]:
startdate = dt.datetime(2021,5,28,22,19)
enddate = startdate + pd.Timedelta(hours=3)

source_location = [67, 19] #longitude, latitude

In [8]:
# Options
event_options = {'04Jan2025': {'date': "2025/01/04 18:27:00",
                               'source': [60, -15]},
                 '17Dec2024': {'date': "2024/12/17 12:53:00",
                               'source': [33, -16]},
                 '08Dec2024': {'date': "2024/12/08 08:50:00",
                               'source': [52, -6]},
                 '03Oct2024': {'date': "2024/10/03 12:08:00",
                               'source': [8, -15]},
                 '01Sep2024': {'date': "2024/09/01 14:44:00",
                               'source': [66, -12]}}

## Show the fleet distribution
For more information on the Solar-MACH tool, see: reflinkhere

NB: If you wish to use BepiColombo data then it will need to downloaded separately and saved to the same folder.

In [10]:
solarmach_table = jtl.solarmach_basic(startdate, data_path, coord_sys='Stonyhurst', source_location=source_location)
display(solarmach_table)        

SolarMACH_28052021_22:19
['SolarMACH_28052021_22:19.csv', 'SolarMACH_28052021_22:19.png', '.ipynb_checkpoints']
WIN


,Spacecraft/Body.1,Stonyhurst longitude (°),Stonyhurst latitude (°),Heliocentric distance (AU),Longitudinal separation to Earth's longitude,Latitudinal separation to Earth's latitude,Vsw,Magnetic footpoint longitude (Stonyhurst),Longitudinal separation between body and reference_long,Longitudinal separation between body's magnetic footpoint and reference_long,Latitudinal separation between body and reference_lat
Spacecraft/Body,,,,,,,,,,,
Earth,Earth,-0.000002,-1.036852,1.013495,0.000000,0.000000,371.290009,68.201091,-67.000002,1.201091,-20.036852
PSP,PSP,61.059695,3.108280,0.685340,61.059698,4.145132,394.720640,104.265955,-5.940305,37.265955,-15.891720
Solar Orbiter,Solar Orbiter,-98.578091,-0.943589,0.950952,-98.578089,0.093263,361.985863,-32.958183,-165.578091,-99.958183,-19.943589
STEREO-A,STEREO-A,-50.395401,-6.234611,0.963644,-50.395399,-5.197760,371.396552,13.897956,-117.395401,-53.102044,-25.234611
BepiColombo,BepiColombo,-110.733366,-3.410311,0.798805,-110.733364,-2.373459,400.000000,-61.011138,-177.733366,-128.011138,-22.410311


## Spacecraft options

Here details the relevant spacecraft, their instruments, which channel(s) is(are) used, and the intercalibration values. The only intercalibration value we have found so far is from [Richardson et al. (2014), page 3064](https://doi.org/10.1007/s11207-014-0524-8) which found SOHO/ERNE-HED 13.8-24.2 MeV proton intensities to be about 1.5 times the STEREO-A/HET proton intensities of the same energy.

In [11]:
## 25-40 MeV Proton channels
#proton_channels = {'PSP': {'instrument': 'EpiHi-HET',
#                           'channels': [8,9],
#                           'intercalibration': 1},
#                   'SOHO': {'instrument': 'ERNE-HED',
#                            'channels': [3,4],
#                            'intercalibration': 1},
#                   'STEREO-A': {'instrument': 'HET',
#                                'channels': [5,8],
#                                'intercalibration': 1},
#                   'Solar Orbiter': {'instrument': 'HET',
#                                     'channels': [19,24],
#                                     'intercalibration': 1}}

# ~14 MeV Proton channels
proton_channels = {'PSP': {'instrument': 'EpiHi-HET',
                           'channels': [3,4],
                           'intercalibration': 1},
                   'SOHO': {'instrument': 'ERNE-HED',
                            'channels': [0],
                            'intercalibration': 0.67},
                   'STEREO-A': {'instrument': 'HET',
                                'channels': [0],
                                'intercalibration': 1},
                   'Solar Orbiter': {'instrument': 'HET',
                                     'channels': [10,12],
                                     'intercalibration': 1}}

Which instruments would you like to include?

In [12]:
spacecraft = ['PSP', 'SOHO', 'STEREO-A', 'Solar Orbiter']
sc_to_plot = spacecraft # if a spacecraft is removed from the above, but you still want it plotted.
intercalibration = False
radial_scaling = False
resampling = '15min'

In [13]:
df = jtl.load_sc_data(spacecraft=spacecraft, 
                      proton_channels=proton_channels,
                      dates=[startdate,enddate],
                      data_path=data_path,
                      intercalibration=intercalibration, 
                      radial_scaling=radial_scaling,
                      resampling=resampling,
                      reference_loc=source_location)

No solar wind speeds defined, trying to obtain measurements...


2025-11-05 15:24:26 - speasy.core.requests_scheduling.request_dispatch - WARNING: CDA server https://cdaweb.gsfc.nasa.gov is down, disabling CDA provider


AttributeError: 'types.SimpleNamespace' object has no attribute 'cda'

In [ ]:
print(pd.date_range(startdate, enddate, freq='15min'))

# Build to one df (synced the times) with headers:
# Layer 1: sc-ins
# Layer 2: Flux, Uncertainty, radial position, longitude.

## Instrument intercalibration - not needed

## Radial Scaling

Using the values presented in [Farwa, et al. (2025)](https://www.aanda.org/articles/aa/abs/2025/01/aa50945-24/aa50945-24.html), which used values for 27-37 MeV protons from [Lario et al. (2006)](https://iopscience.iop.org/article/10.1086/508982) (for ~100 keV electrons, [Rodríguez-García et al. (2023)](https://www.aanda.org/10.1051/0004-6361/202244553) is used).

The scaled intensity is calculated as $I_{1 au} = I \cdot R^{a\pm b}$, where $R$ is the radial distance, $I$ is the original intensity, and (for protons specifically) the scaling factors are given as $a \pm b = 1.97 \pm 0.27$.

To calculate the scaled uncertainty, we use the following procedure:
1. Calculate the boundary limits for the intensity calculation (e.g. the result should be $I_{-\alpha}^{+\beta}$).
2. Find the higher boundary limit, as long as it is < the nominal value (e.g. $\beta$).
3. Calculate the scaled uncertainty value: $\Delta I_{1 au} = \Delta I \cdot R^a$.
4. Combine both to get a final uncertainty value: $\Delta I_{1 au, final} = \sqrt{(\beta)^2 + (\Delta I_{1 au})^2}$.

NB: Check that this final uncertainty is still less than the intensity value!